In [2]:
# Dependencies
from bs4 import BeautifulSoup, SoupStrainer
import requests
import pandas as pd
from pprint import pprint
import re
import time

In [3]:
house_dict = []

for page in range(17, 21):
    linked = 'https://www.zillow.com/ca/houses/' + str(page) + '_p/?searchQueryState={"pagination":{"currentPage":' + str(page) + '},"usersSearchTerm":"ca","mapBounds":{"west":-130.9521558125,"east":-107.6611401875,"south":28.570940002914515,"north":45.333552921012235},"regionSelection":[{"regionId":9,"regionType":2}],"isMapVisible":true,"mapZoom":5,"filterState":{"sortSelection":{"value":"globalrelevanceex"},"isCondo":{"value":false},"isMultiFamily":{"value":false},"isManufactured":{"value":false},"isLotLand":{"value":false},"isTownhouse":{"value":false},"isApartment":{"value":false},"isComingSoon":{"value":false},"isAuction":{"value":false},"doz":{"value":"90"}},"isListVisible":true}'
    user_agent = 'Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1'
    headers = {'User-Agent': user_agent}
    response = requests.get(linked, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    results = soup.find_all('article', class_='list-card')
    
    # Loop through returned results
    for result in results:
        # Error handling
        try:
            # Identify and return housing weblink
            house_website = str(result.a.get('href'))
            print(house_website)
            # Identify and return house data
            listing_type = result.find('div', class_="list-card-footer").text;
            price = result.find('div', class_="list-card-price").text
#             print(price)
            full_address = result.find('address', class_="list-card-addr").text
            street = result.find('address', class_="list-card-addr").text.split(",")[0]
            try:
                city = result.find('address', class_="list-card-addr").text.split(",")[1].replace(" ", "")
            except IndexError:
                city = 'null'
            try:
                state = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[1]
            except IndexError:
                state = 'null'
            try:
                zip_code = result.find('address', class_="list-card-addr").text.split(",")[2].split(" ")[2]
            except IndexError:
                zip_code = 'null'
            bed = result.find('ul', class_="list-card-details").find('li').text.split(" ")[0]
            bath = result.find('ul', class_="list-card-details").select('li')[1].text.split(" ")[0]
            house_square_feet = result.find('ul', class_="list-card-details").select('li')[2].text.split(" ")[0]
            # Loop inside each house website and get more house data
            loop_response = requests.get(house_website, headers=headers)
            loop_soup = BeautifulSoup(loop_response.text, 'html.parser')
            loop_result1 = loop_soup.find('ul', class_="ds-home-fact-list")
            loop_result2 = loop_soup.findAll('li', class_="ds-home-fact-list-item")

            lot = 'null';
            house_type = 'null';
            year_build = 'null';
            parking = 'null';
            hoa = 'null';
            lot_conversion_to_sqft = 'null';
            zillow_price_per_sqft = 'null';
            
            for content in loop_result2:
                if "Lot" in content.text.split(":")[0]:
                    lot = content.text.split(":")[1];
                if "Type" in content.text.split(":")[0]:
                    house_type = content.text.split(":")[1];
                if "Year built" in content.text.split(":")[0]:
                    year_build = content.text.split(":")[1];
                if "Parking" in content.text.split(":")[0]:
                    parking = content.text.split(":")[1];
                if "HOA" in content.text.split(":")[0]:
                    hoa = content.text.split(":")[1];
                if "Price/sqft" in content.text.split(":")[0]:
                    zillow_price_per_sqft = content.text.split(":")[1];
                    
            try:
                if (str(lot.split(" ")[1]) == "sqft"):
                    lot_conversion_to_sqft = float(lot.split(" ")[0].replace(',', ""));
                elif ((str(lot.split(" ")[1]) == "acres") or (str(lot.split(" ")[1]) == "acre") or (str(lot.split(" ")[1]) == "Acres") or (str(lot.split(" ")[1]) == "Acre")):
                    lot_conversion_to_sqft = round(float(lot.split(" ")[0].replace(',', "")) * 43560, 2);
                elif (lot == "No Data"):
                    lot_conversion_to_sqft = 'null'
                else:
                    lot_conversion_to_sqft = 'null'
            except IndexError:
                lot_conversion_to_sqft = 'null'
                
            try:
                zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[0].text
            except:
                zestimate = 'null'
            try:
                rent_zestimate = loop_soup.findAll('span', class_="Text-c11n-8-33-0__aiai24-0 LoSpP")[1].text
                
            except:
                rent_zestimate = 'null'
            if (zestimate[0] != '$'):
                zestimate = 'null';
            print('zestimate', zestimate)
            print('rent zestimate', rent_zestimate)

            # Store data to house object if all elements are available
#             if (house_website and listing_type and price and full_address and street and city and state and zip_code and bed and bath and house_square_feet and house_type and year_build):
            house_obj = {
                "Listing_type": listing_type,
                "Price": price,
                "Full_address": full_address,
                "Street": street,
                "City": city,
                "State": state,
                "Zip_code": zip_code,
                "Bed": bed,
                "Bath": bath,
                "Square_Feet": house_square_feet,
                "Lot": lot,
                "Lot_conversion_to_sqft": lot_conversion_to_sqft,
                "House_type": house_type,
                "Year_build": year_build,
                "Parking": parking,
                "HOA": hoa,
                "Zestimate": zestimate,
                "Rent_zestimate": rent_zestimate,
                "Zillow_price/sqft": zillow_price_per_sqft,
                "House_website": house_website
            }

            #Continue to add data into collection house_dict
            house_dict.append(house_obj)
            time.sleep(3)
            
        except AttributeError as e:
            continue

print(house_dict)

[]


In [19]:
house_data = pd.DataFrame(house_dict)
house_data

,Listing_type,Price,Full_address,Street,City,State,Zip_code,Bed,Bath,Square_Feet,Lot,Lot_conversion_to_sqft,House_type,Year_build,Parking,HOA,Zestimate,Rent_zestimate,Zillow_price/sqft,House_website
0,KELLER WILLIAMS REALTY IRVINE,"$770,000","11924 Gurley Ave, Downey, CA 90242",11924 Gurley Ave,Downey,CA,90242,4,3,"1,604","13,094 sqft",13094,SingleFamily,1947,"Driveway, Garage, Oversized",null,"$765,286","$3,000/mo",$480,https://www.zillow.com/homedetails/11924-Gurle...
1,,"$230,000","3862 E Norwich Ave, Fresno, CA 93726",3862 E Norwich Ave,Fresno,CA,93726,4,2,"1,306","6,599 sqft",6599,SingleFamily,1968,"Garage - Attached,",null,"$239,068","$1,500/mo",$176,https://www.zillow.com/homedetails/3862-E-Norw...
2,KING REALTY GROUP INC,"$624,800","2444 E Norwich St, Ontario, CA 91761",2444 E Norwich St,Ontario,CA,91761,4,3,"2,241","5,501 sqft",5501,SingleFamily,2017,Garage - Attached,$90 monthly,"$607,274","$2,700/mo",$279,https://www.zillow.com/homedetails/2444-E-Norw...
3,REDFIN,"$449,900","4281 Tola Ct, Riverside, CA 92509",4281 Tola Ct,Riverside,CA,92509,4,2,"1,288","16,552 sqft",16552,SingleFamily,1972,Garage,null,"$450,108","$2,345/mo",$349,https://www.zillow.com/homedetails/4281-Tola-C...
4,"HOMESMART, EVERGREEN REALTY","$849,000","22501 Auburn Dale Dr, Lake Forest, CA 92630",22501 Auburn Dale Dr,LakeForest,CA,92630,3,2,"1,329","7,701 sqft",7701,SingleFamily,1977,"Driveway Level, Garage",null,"$883,095","$3,000/mo",$639,https://www.zillow.com/homedetails/22501-Aubur...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,COMPASS,"$1,179,000","8511 Cole Crest Dr, Los Angeles, CA 90046",8511 Cole Crest Dr,LosAngeles,CA,90046,4,3,"1,515","4,319 sqft",4319,SingleFamily,1961,Driveway,null,"$1,233,712","$3,595/mo",$778,https://www.zillow.com/homedetails/8511-Cole-C...
156,,"$489,999","5913 Engstrom Dr, Riverbank, CA 95367",5913 Engstrom Dr,Riverbank,CA,95367,4,3,"2,532","5,458 sqft",5458,SingleFamily,2003,"Garage - Attached,",null,"$517,998","$2,500/mo",$194,https://www.zillow.com/homedetails/5913-Engstr...
157,,"$465,000","1451 Bronson Ave, Modesto, CA 95350",1451 Bronson Ave,Modesto,CA,95350,4,2,"1,860","7,714 sqft",7714,SingleFamily,1952,"Garage - Attached,",null,"$416,199","$1,975/mo",$250,https://www.zillow.com/homedetails/1451-Bronso...
158,EXCELLENCE RE SOCAL PROPERTIES,"$179,900","99620 Tripoli Dr, Mecca, CA 92254",99620 Tripoli Dr,Mecca,CA,92254,2,1,"1,266","33,106 sqft",33106,SingleFamily,1949,Driveway,$350 annually,"$2,100/mo",null,$142,https://www.zillow.com/homedetails/99620-Tripo...


In [20]:
# Export file as a CSV, without the Pandas index, but with the header
house_data.to_csv("Output/Zillow/Q1-2021_Single_Family_Data_page17-20.csv", index=False, header=True)